In [1]:
import time
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM
from matplotlib import pyplot as plt
from math import sqrt

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# input part
def get_data():
    data = pd.read_csv('data.csv')
    df = pd.DataFrame(data)
    df.drop(df.columns[[0, 3, 7, 8]], axis=1, inplace=True)
    return df

In [ ]:
# cleaning and spliting part
def load_data(data, seq_len):
    var_count = len(data.columns)
    data = data.values
    sequence_length = seq_len+1
    result = []
    for i in range(len(data)-sequence_length):
        result.append(data[i:i+sequence_length])
    result = np.array(result)
    row = round(0.8*result.shape[0])
    train = result[:int(row), :]
    x_train = train[:, :-1]
    y_train = train[:, -1][:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1][:, -1]
    x_train = np.reshape(
        x_train, (x_train.shape[0], x_train.shape[1], var_count))
    x_test = np.reshape(
        x_test, (x_test.shape[0], x_test.shape[1], var_count))
    print(x_train[:3])
    print(y_train[:3])
    return [x_train, y_train, x_test, y_test]

In [ ]:
# building of untrained data
def build_model(layers):
    model = Sequential()
    model.add(LSTM(50, input_shape=(
        layers[1], layers[0]), return_sequences=False))
    model.add(Dense(1, activation="relu", kernel_initializer="uniform"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop", metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

In [ ]:
df = get_data()
print(df)

In [ ]:
values = df.values
print(values)

In [ ]:
minima = np.amin(values[:, -1])
maxima = np.amax(values[:, -1])
scaling_parameter = maxima-minima

In [ ]:
values[:, 4] = (values[:, 4] - minima) / scaling_parameter
values[:, 0] = (values[:, 0]-np.amin(values[:, 0])) / \
    (np.amax(values[:, 0])-np.amin(values[:, 0]))
values[:, 1] = (values[:, 1]-np.amin(values[:, 1])) / \
    (np.amax(values[:, 1])-np.amin(values[:, 1]))
values[:, 2] = (values[:, 2]-np.amin(values[:, 2])) / \
    (np.amax(values[:, 2])-np.amin(values[:, 2]))
values[:, 3] = (values[:, 3]-np.amin(values[:, 3])) / \
    (np.amax(values[:, 3])-np.amin(values[:, 3]))

In [ ]:
df = pd.DataFrame(values)
window = 5
x_train, y_train, x_test, y_test = load_data(df[::-1], window)

In [ ]:
print(y_test)

In [ ]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_test", x_test.shape)
print("y_test", y_test.shape)

In [ ]:
model = build_model([5, window, 1])

In [ ]:
print(x_train.shape)

In [ ]:
# training part, can also change batch size and epoch to get more better result
model.fit(
    x_train,
    y_train,
    batch_size=1024,
    epochs=100,
    validation_split=0.2,
    verbose=2
)

In [ ]:
# function to calculate error
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred)/y_true))*100

In [ ]:
# testing part
predicted = model.predict(x_test)
predicted = (predicted*scaling_parameter)+minima
actual = y_test
actual = (actual*scaling_parameter)+minima
print(actual)
print(predicted)

In [ ]:
mape = sqrt(mean_absolute_percentage_error(predicted, actual))
print("Test MAPE: %0.3f" % mape)

In [ ]:
# result presenting part
plt.plot(actual[:24], label='Actual', color="blue")
plt.plot(predicted[:24], label='Predicted', color="red")
plt.ylabel("Load(MW)")
plt.title("Actual VS Predicted results")
plt.legend()
plt.show()